# Computation

## General concepts and mechanisms

### Overview

Binary operations between data arrays or datasets behave as follows:

| Property | Action |
| --- | --- |
| coord | compare, abort on mismatch |
| data | apply operation |
| mask | combine with `or` |
| attr | typically ignored or dropped |

In the special case of in-place operations such as `+=` or `*=` scipp preserves existing attributes and ignores attributes of the right-hand-side.

### Dimension matching and transposing

Operations "align" variables based on their dimension labels.
That is, an operation between two variables that have a transposed memory layout behave correctly:

In [1]:
import numpy as np
import scipp as sc

a = sc.Variable(values=np.random.rand(2, 4),
                variances=np.random.rand(2, 4),
                dims=['x', 'y'],
                unit=sc.units.m)
b = sc.Variable(values=np.random.rand(4, 2),
                variances=np.random.rand(4, 2),
                dims=['y', 'x'],
                unit=sc.units.s)
a/b

<scipp.Variable>          float64    [m s^-1]         (x, y)  [0.420123, 0.144385, ..., 0.211529, 0.892483]  [1.010567, 0.168984, ..., 0.735612, 9.789578]

### Propagation of uncertainties

If variables have variances, operations correctly propagate uncertainties (the variances), in contrast to a naive implementation using numpy:

In [2]:
result = a/b
result.values

array([[ 0.4201231 ,  0.14438539,  0.49299548, 10.82728823],
       [ 0.96419514,  0.65866116,  0.21152862,  0.89248294]])

In [3]:
a.values/np.transpose(b.values)

array([[ 0.4201231 ,  0.14438539,  0.49299548, 10.82728823],
       [ 0.96419514,  0.65866116,  0.21152862,  0.89248294]])

In [4]:
result.variances

array([[1.01056705e+00, 1.68983904e-01, 1.75703456e+00, 2.79029338e+04],
       [1.48319764e+00, 2.13830006e+00, 7.35611835e-01, 9.78957842e+00]])

In [5]:
a.variances/np.transpose(b.variances)

array([[1.30176508e+00, 7.26395405e-02, 6.91062909e-01, 5.17416814e-01],
       [9.57829656e-02, 1.03037389e+00, 4.10002195e-01, 1.30901990e+02]])

The implementation assumes uncorrelated data and is otherwise based on, e.g., [Wikipedia: Propagation of uncertainty](https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Example_formulae>).
See also [Propagation of uncertainties](../python-reference/error-propagation.rst) for the concrete equations used for error propagation.

### Broadcasting

Missing dimensions in the operands are automatically broadcast.
Consider:

In [6]:
var_xy = sc.Variable(dims=['x', 'y'], values=np.arange(6).reshape((2,3)))
print(var_xy.values)

[[0 1 2]
 [3 4 5]]


In [7]:
var_y = sc.Variable(dims=['y'], values=np.arange(3))
print(var_y.values)

[0 1 2]


In [8]:
var_xy -= var_y
print(var_xy.values)

[[0 0 0]
 [3 3 3]]


Since `var_y` did not depend on dimension `'x'` it is considered as "constant" along that dimension.
That is, the *same* `var_y` values are subtracted *from all slices of dimension `'x'`* in `var_xy`.

Coming back to our original variables `a` and `b`, we see that broadcasting integrates seamlessly with slicing and transposing:

In [9]:
a.values

array([[0.33428308, 0.10626499, 0.35197483, 0.59522048],
       [0.77607305, 0.50735167, 0.10619917, 0.24692973]])

In [10]:
a -= a['x', 1]
a.values

array([[-0.44178998, -0.40108669,  0.24577565,  0.34829075],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

Both operands may be broadcast, creating an output with the combination of input dimensions:

In [11]:
sc.show(a['x', 1])
sc.show(a['y', 1])
sc.show(a['x', 1] + a['y', 1])

Note that in-place operations such as `+=` will never change the shape of the left-hand-side.
That is only the right-hand-side operation can be broadcast, and the operation fails of a broadcast of the left-hand-side would be required.

### Units

Units are required to be compatible:

In [12]:
try:
    a + b
except Exception as e:
    print(str(e))

Cannot add m and s.


### Coordinate and name matching

In operations with datasets, data items are paired based on their names when applying operations to datasets.
Operations fail if names do not match:

- In-place operations such as `+=` accept a right-hand-side operand that omits items that the left-hand-side has.
  If the right-hand-side contains items that are not in the left-hand-side the operation fails.
- Non-in-place operations such as `+` return a new dataset with items from the intersection of the inputs.

Coords are compared in operations with datasets or data arrays (or items of datasets).
Operations fail if there is any mismatch in coord or label values.

In [13]:
d1 = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2)),
     'c': sc.Variable(dims=['x'], values=np.random.rand(2)),
     'd': sc.Variable(1.0)},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})
d2 = sc.Dataset(
    {'a': sc.Variable(dims=['x', 'y'], values=np.random.rand(2, 3)),
     'b': sc.Variable(dims=['y', 'x'], values=np.random.rand(3, 2))},
    coords={
        'x': sc.Variable(['x'], values=np.arange(2.0), unit=sc.units.m),
        'y': sc.Variable(['y'], values=np.arange(3.0), unit=sc.units.m)})

In [14]:
d1 += d2

In [15]:
try:
    d2 += d1
except Exception as e:
    print(str(e))

Expected <scipp.Dataset>
Dimensions: {{x, 2}, {y, 3}}
Coordinates:
    x                         float64    [m]              (x)  [0.000000, 1.000000]
    y                         float64    [m]              (y)  [0.000000, 1.000000, 2.000000]
Data:
    a                         float64    [dimensionless]  (x, y)  [0.702142, 0.830409, ..., 0.366502, 0.807485]
    b                         float64    [dimensionless]  (y, x)  [0.914519, 0.622650, ..., 0.196682, 0.304142]

 to contain d.


In [16]:
d3 = d1 + d2
for name in d3:
    print(name)

a
b


In [17]:
d3['a'] -= d3['b'] # transposing
d3['a'] -= d3['x', 1]['b'] # broadcasting
try:
    d3['a'] -= d3['x', 1:2]['b'] # fail due to coordinate mismatch
except Exception as e:
    print(str(e))

Expected coords to match.


## Arithmetics

The arithmetic operations `+`, `-`, `*`, and `/` and their in-place variants `+=`, `-=`, `*=`, and `/=` are available for variables, data arrays, and datasets.
They can also be combined with [slicing](slicing.ipynb).

## Trigonometrics

Trigonometric functions like `sin` are supported for variables.
Units for angles provide a safeguard and ensure correct operation when working with either degree or radian:

In [18]:
rad = 3.141593*sc.units.rad
deg = 180.0*sc.units.deg
print(sc.sin(rad))
print(sc.sin(deg))
try:
    rad + deg
except Exception as e:
    print(str(e))

    <scipp.Variable>          float64    [dimensionless]  ()  [-0.000000]

    <scipp.Variable>          float64    [dimensionless]  ()  [0.000000]

Cannot add rad and deg.


## Other

See the [list of free functions](../python-reference/api.rst#free-functions) for an overview of other available operations.